---
 
Author : Halisson S. Gomides - halisson.gomides@gmail.com

Date : 22/01/2021

---

### Conjunto de dados House Sales in King County, USA
>https://www.kaggle.com/harlfoxem/housesalesprediction


### Novas perguntas do CEO:
>1. Qual a data do imóvel mais antigo no portfolio?
>2. Quantos imóveis possuem o número máximo de andares?
>3. Criar uma classificação para os imóveis, separando-os em baixo e alto padrão, de acordo com o preço.
>> Acima de 540.000 -> alto padrão
>> Abaixo de 540.000 -> baixo padrão
>4. Relatório ordenado pelo preço e contendo as seguintes informações:
>> id do imóvel, data que o imóvel ficou disponível para compra, número de quartos, tamanho tota do terreno, preço, a classificação do imóvel (alto e baixo padrão)
>5. Mapa indicando onde as casas estão localizadas geograficamente

In [ ]:
import pandas as pd

In [ ]:
dfhouses = pd.read_csv('dados/originais/kc_house_data.csv.zip', parse_dates=['date'], infer_datetime_format=True)

In [ ]:
dfhouses.head()

In [ ]:
dfhouses.info()

In [ ]:
# Verificando se existem indices duplicados

for k,v in dfhouses['id'].value_counts().to_dict().items():
    if v > 1: 
        print(k, v)
        print(dfhouses.loc[dfhouses['id'] == k])

        

In [ ]:
# Função para apresentar o resultado (pergunta / resposta)
from typing import Union

def resultado(iq:int, resposta:Union[int,float,list], txtcompl:str = '') -> None:
    '''
    Função para apresentar o resultado (pergunta / resposta)
    '''
    
    list_questoes = [
        'Qual a data do imóvel mais antigo no portfolio?',
        'Quantos imóveis possuem o número máximo de andares?',
        'Criar uma classificação para os imóveis, separando-os em baixo e alto padrão, de acordo com o preço.',
        'Relatório ordenado pelo preço.',
        'Mapa indicando onde as casas estão localizadas geograficamente.']
    
    if type(resposta) is list:
        resposta = ', '.join(map(str, resposta))
        
    print(f'{iq+1}. {list_questoes[iq]}\n Resp.: {txtcompl} {resposta}')
    
    


In [ ]:
# Removendo as casas com IDs duplicados, mantendo apenas o registro com a data mais recente

df_houses = dfhouses.sort_values(by=['id', 'date'])
# df_order.head(10)
df_houses.drop_duplicates(subset=['id'], keep='last', inplace=True)

# testando o resultado com um id que eu sei que é duplicado
df_houses.query('id == 795000620')

In [ ]:
# Qual a data do imóvel mais antigo no portfolio?

resultado(0, df_houses.nsmallest(1, 'date')['date'].tolist()[0].strftime('%d/%m/%Y'))

In [ ]:
# Quantos imóveis possuem o número máximo de andares?

resultado(1, df_houses.loc[df_houses['floors'] == df_houses['floors'].max()].shape[0])

In [ ]:
# Criar uma classificação para os imóveis, separando-os em baixo e alto padrão, de acordo com o preço.

df_houses['padrao'] = df_houses['price'].apply(lambda preco: 'baixo_padrao' if preco < 540000 else 'alto_padrao')

In [ ]:
# nº de casas de alto padrão
lst_padrao = [df_houses.query('padrao == "alto_padrao"').shape[0], df_houses.query('padrao == "baixo_padrao"').shape[0]]
lst_padrao[0]

# nº de casas de baixo padrão
lst_padrao[1]

resultado(2, lst_padrao, 'nº de casas de alto padrão e baixo padrão respectivamente.')

In [ ]:
# Relatório ordenado pelo preço.

# id do imóvel, 
# data que o imóvel ficou disponível para compra, 
# número de quartos, 
# tamanho total do terreno, 
# preço, 
# a classificação do imóvel (alto e baixo padrão)

lst_columns = ['id', 'date', 'bedrooms', 'sqft_lot', 'price', 'padrao']
df_houses.sort_values(by='price')[lst_columns].to_csv('dados/processados/ex02_relatorioCEO.csv', index=False)


In [ ]:
# Mapa indicando onde as casas estão localizadas geograficamente.

!pip install plotly

In [ ]:
import plotly.express as px

In [ ]:
lst_columns = ['id', 'lat', 'long', 'price']
mapa = px.scatter_mapbox( df_houses[lst_columns], lat='lat', lon='long',
                 hover_name='id',
                 hover_data=['price'],
                 color_discrete_sequence=['fuchsia'],
                 size='price',
                 height=300)
mapa.update_layout(mapbox_style='open-street-map')
mapa.update_layout(height=600, margin={'r':0, 't':0, 'l':0, 'b':0})
mapa.show()

mapa.write_html('dados/processados/ex02_mapa_house_rocket.html')